**Question 1**: By default are django signals executed synchronously or asynchronously?
Please support your answer with a code snippet that conclusively proves your stance.
The code does not need to be elegant and production ready, we just need to understand your logic.

**Answer**: By default, Django signals are executed synchronously. This means that when a signal is sent, the connected signal handlers are executed immediately and block the sender until they are completed.

In [ ]:
from time import sleep
from django.contrib.auth.models import User
from django.db.models.signals import post_save
from django.dispatch import receiver


@receiver(post_save, sender=User)
def user_saved_handler(sender, instance, **kwargs):
    """
    Signal handler that is called after a User instance is saved.

    Args:
        sender (Model): The model class that sent the signal.
        instance (User): The instance of the model that was saved.
        **kwargs: Additional keyword arguments.
    """
    print("Signal handler started")
    sleep(5)  # Simulate a delay
    print("Signal handler finished")


def save_user():
    """
    Create and save a new User instance.

    This function simulates saving a User instance
    and prints messages before and after the save operation.
    """
    user = User(username="test_user")
    print("Before saving user")
    user.save()
    print("After saving user")

save_user()

**Question 2**: Do django signals run in the same thread as the caller?
Please support your answer with a code snippet that conclusively proves your stance.
The code does not need to be elegant and production ready, we just need to understand your logic.

**Answer**: Yes, Django signals run in the same thread as the caller by default.
This means that the signal handler runs in the same thread that initiated the signal.

In [ ]:
import threading
from django.contrib.auth.models import User
from django.db.models.signals import post_save
from django.dispatch import receiver


@receiver(post_save, sender=User)
def user_saved_handler(sender, instance, **kwargs):
    """
    Signal handler that is called after a User instance is saved.

    Args:
        sender (Model): The model class that sent the signal.
        instance (User): The instance of the model that was saved.
        **kwargs: Additional keyword arguments.
    """
    print(f"Signal handler thread ID: {threading.get_ident()}")


def save_user():
    """
    Create and save a new User instance.

    This function simulates saving a User instance and
    prints the thread ID of the caller before saving.
    """
    user = User(username="test_user")
    print(f"Caller thread ID: {threading.get_ident()}")
    user.save()

save_user()

**Question 3**: By default do django signals run in the same database transaction as the caller? 
Please support your answer with a code snippet that conclusively proves your stance. 
The code does not need to be elegant and production ready, we just need to understand your logic.

**Answer**: Yes, by default, Django signals run in the same database transaction as the caller. Specifically, signals like `post_save` and `post_delete` are executed after the changes have been made in the database, within the same transaction. If an exception is raised in the signal handler, it will not affect the already committed database changes, whereas the `pre_save` and `pre_delete` signals occur before the transaction is committed. To conclusively prove that Django signals run in the same transaction, you can use a signal handler that raises an exception and verify whether the database changes are rolled back. Here, we'll use a `post_save` signal.

In [ ]:
from django.db import transaction, IntegrityError
from django.db.models.signals import post_save
from django.dispatch import receiver
from django.contrib.auth.models import User


@receiver(post_save, sender=User)
def user_saved_handler(sender, instance, **kwargs):
    """
    Signal handler that is called after a User instance is saved.

    Args:
        sender (Model): The model class that sent the signal.
        instance (User): The instance of the model that was saved.
        **kwargs: Additional keyword arguments.

    Raises:
        Exception: Simulates an error in the signal handler.
    """
    print("Signal handler started")
    raise Exception("Exception in signal handler")


def save_user():
    """
    Create and save a new User instance within a transaction.

    This function attempts to create and save a User instance
    inside an atomic transaction. If an exception is raised, it
    catches the exception and prints an error message.
    """
    try:
        with transaction.atomic():
            user = User.objects.create(username="test_user")
            print(f"User saved with ID: {user.id}")
    except Exception as e:
        print(f"Exception caught: {e}")

save_user()

user_exists = User.objects.filter(username="test_user").exists()
print(f"User exists after transaction: {user_exists}")

**Topic: Custom Classes in Python**

***Description***: You are tasked with creating a Rectangle class with the following requirements:

- An instance of the `Rectangle` class requires `length:int` and `width:int` to be initialized.
- We can iterate over an instance of the `Rectangle class` 
- When an instance of the Rectangle class is iterated over, we first get its length in the format: {'length': <VALUE_OF_LENGTH>} followed by the width {width: <VALUE_OF_WIDTH>}

In [1]:
class Rectangle:
    def __init__(self, length: int, width: int):
        self.length = length
        self.width = width

    def __iter__(self):
        yield {"length": self.length}
        yield {"width": self.width}


rectangle = Rectangle(10, 5)

for dimension in rectangle:
    print(dimension)

{'length': 10}
{'width': 5}
